In [1]:
import pandas as pd
import numpy as np
import requests as rq

In [9]:
API_KEY = "RGAPI-2c16311f-5cfc-4724-8180-b3f72c943c3e"
base_diamond = "https://br1.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/"
base_challenger = "https://br1.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5/"
base_master = "https://br1.api.riotgames.com/lol/league/v4/masterleagues/by-queue/RANKED_SOLO_5x5"
base_grand_master = "https://br1.api.riotgames.com/lol/league/v4/grandmasterleagues/by-queue/RANKED_SOLO_5x5"

In [62]:
def getPlayerInfos(string):
    response = rq.get(string)
    return response.json()

In [61]:
def getLowEloInfos(json):
    df = pd.DataFrame(json)
    df["tier"] = df['tier'] + df["rank"]
    df.drop(["leagueId","queueType","rank","miniSeries"], axis=1,inplace=True)
    return df   

In [89]:
def getHighEloInfos(json):
    df_aux = pd.DataFrame(json["entries"])
    df_aux.drop("rank", axis=1, inplace=True)
    df_aux["tier"] = "MASTER"
    df_aux = df_aux.reindex(
        columns=['tier', 'summonerId', 'summonerName', 
                 'leaguePoints', 'wins', 'losses','veteran',
                 'inactive', 'freshBlood', 'hotStreak'])
    return df_aux

In [94]:
divisions = ["I","II","III","IV"]
tiers = ["DIAMOND","MASTER","GRANDMASTER","CHALLENGER"]
df = pd.DataFrame()
for i in tiers:
    if i == "DIAMOND":
        for j in divisions:
            string = base_diamond + i + "/" + j + "?api_key=" + API_KEY
            df_aux = getLowEloInfos(getPlayerInfos(string))
            df = df.append(df_aux,ignore_index = True)
    
    elif i == "MASTER":
        string = base_master + "?api_key=" + API_KEY
        df_aux = getHighEloInfos(getPlayerInfos(string))
        df = df.append(df_aux,ignore_index = True)
    
    
    elif i == "GRANDMASTER":
        string = base_grand_master + "?api_key=" + API_KEY
        df_aux = getHighEloInfos(getPlayerInfos(string))
        df = df.append(df_aux,ignore_index = True)
    
    else:
        string = base_challenger + "?api_key=" + API_KEY
        df_aux = getHighEloInfos(getPlayerInfos(string))
        df = df.append(df_aux,ignore_index = True)
        

In [98]:
df.to_csv("usersBasicInfos.csv")

#### Agora vamos enriquecer esses dados com mais informações 

In [10]:
df.columns

Index(['leagueId', 'queueType', 'tier', 'rank', 'summonerId', 'summonerName',
       'leaguePoints', 'wins', 'losses', 'veteran', 'inactive', 'freshBlood',
       'hotStreak', 'miniSeries'],
      dtype='object')

In [6]:
api_key = 'RGAPI-1270892c-af42-42ed-ad37-00ee4675c997'
watcher = LolWatcher(api_key)
my_region = 'br1'

me = watcher.summoner.by_name(my_region, 'rllima')
print(me)

{'id': 'qflJMcN2-FIcY-GZrVMB7iWWhsxxIYatZQbAfz0ye57128I', 'accountId': 'XtKl6pm7xfo2gxXM589ypwrT4Amm8iN3YHl369-_JHms9S0', 'puuid': 'DR0QmgC8OjqNl9j1Y8Bme8OGO6dT8OcsrxDT_e_xWUzQgmG4CS4kKCst3UZGbiEbtH99oATTOLaJkA', 'name': 'rllima', 'profileIconId': 4629, 'revisionDate': 1599069432000, 'summonerLevel': 181}
